In [ ]:
import pandas as pd

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-11.parquet

In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv


In [ ]:
df_zones = pd.read_csv("taxi_zone_lookup.csv")
df_zones.head()

In [ ]:
df_trips = pd.read_parquet('green_tripdata_2025-11.parquet')

In [ ]:
df_trips.head()

In [ ]:
len(df_trips.query('trip_distance <= 1.0 and lpep_pickup_datetime >= "2025-11-01" and lpep_dropoff_datetime < "2025-12-01"'))

In [ ]:
df_trips.head()

In [ ]:
df_trips.sort_values(by = ['trip_distance'], ascending=False).query('trip_distance < 100')

In [8]:
df_trips.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [9]:
df_zones.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [22]:
df_full = pd.merge(df_zones, df_trips, left_on='LocationID', right_on='PULocationID', how='left')
filtered_df = df_full.query('lpep_pickup_datetime >= "2025-11-18" and lpep_dropoff_datetime < "2025-11-19"').sort_values(by=['total_amount'], ascending=False)
filtered_df.groupby('Zone')['total_amount'].sum().sort_values(ascending=False)

Zone
East Harlem North           9281.92
East Harlem South           6609.15
Central Park                2378.79
Washington Heights South    2139.05
Morningside Heights         2100.59
                             ...   
Upper West Side South         16.00
Upper West Side North         16.00
West Village                  16.00
Bronxdale                     13.57
Brooklyn Navy Yard             0.00
Name: total_amount, Length: 136, dtype: float64

In [19]:
filtered_df.groupby('Zone')['total_amount'].sum().sort_values(ascending=False)

Zone
East Harlem North           9281.92
East Harlem South           6609.15
Central Park                2378.79
Washington Heights South    2139.05
Morningside Heights         2100.59
                             ...   
Upper West Side South         16.00
Upper West Side North         16.00
West Village                  16.00
Bronxdale                     13.57
Brooklyn Navy Yard             0.00
Name: total_amount, Length: 136, dtype: float64

In [28]:
df_full.head()
res = df_full.query('Zone == "East Harlem North" and lpep_pickup_datetime >= "2025-11-01" and lpep_dropoff_datetime <"2025-12-01"').sort_values(['tip_amount'],ascending=False).head(1)
res['DOLocationID'].values[0]

np.float64(263.0)

In [37]:
df_full2 = pd.merge(df_zones, df_trips, left_on='LocationID', right_on='PULocationID', how='left')
res = df_full2.query('Zone == "East Harlem North"').sort_values(['tip_amount'],ascending=False).head(1)
res_val = res['DOLocationID'].values[0]

df_zones[df_zones['LocationID'] == res_val]



,LocationID,Borough,Zone,service_zone
262,263,Manhattan,Yorkville West,Yellow Zone


In [38]:
# 1. Merge on PULocationID (Pickup) so 'Zone' becomes the Pickup Name
df_full_pickup = pd.merge(df_zones, df_trips, left_on='LocationID', right_on='PULocationID', how='left')

# 2. Filter for trips STARTING in East Harlem North
# (Don't forget the date filter! The question asks for November 2025)
target_trip = df_full_pickup.query(
    'Zone == "East Harlem North" and lpep_pickup_datetime >= "2025-11-01" and lpep_pickup_datetime < "2025-12-01"'
).sort_values('tip_amount', ascending=False).head(1)

# 3. Get the Drop Off ID of the winner
drop_off_id = target_trip['DOLocationID'].values[0]

# 4. Look up the name of that Drop Off zone
result_name = df_zones.loc[df_zones['LocationID'] == drop_off_id, 'Zone'].values[0]
print(result_name)

Yorkville West
